In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url, usecols=[1], engine='python')
df.head()


,Passengers
0,112
1,118
2,132
3,129
4,121


In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df.values)

In [5]:
training_data_len = int(np.ceil(len(scaled_data) * 0.8))
train_data = scaled_data[0:training_data_len, :]
test_data = scaled_data[training_data_len - 60: , :]

In [6]:
def create_dataset(data , time_step = 1):
    X , Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step) , 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X) , np.array(Y)    

In [7]:
time_step = 60
X_train ,  y_train = create_dataset(train_data , time_step)
X_test , y_test = create_dataset(test_data , time_step)


In [8]:
X_train = X_train.reshape(X_train.shape[0] , X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [9]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')


In [10]:
model.fit(X_train, y_train, batch_size=1, epochs=1)



55/55 [==============================] - 3s 12ms/step - loss: 0.0259


In [11]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)


1/1 [==============================] - 0s 498ms/step


In [12]:
predictions

array([[393.31976],
       [397.69577],
       [400.78192],
       [401.87827],
       [401.85965],
       [401.36377],
       [400.19836],
       [399.706  ],
       [399.58142],
       [400.2005 ],
       [402.29968],
       [406.80252],
       [413.22742],
       [419.3283 ],
       [424.04617],
       [426.7468 ],
       [428.53674],
       [429.75528],
       [430.05988],
       [430.16495],
       [430.85126],
       [432.1762 ],
       [435.06027],
       [440.52618],
       [447.5626 ],
       [454.00687],
       [459.03647]], dtype=float32)

In [13]:
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 418.54374085906136
